In [15]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from dotenv import load_dotenv

#### Zero Shot Prompting

In [ ]:
load_dotenv()
def generate_privacy_policies():
	"""
	Generates two privacy policies:
	1. A vague zero-shot version
	2. A specific zero-shot version
	Returns both responses as a dictionary.
	"""

	llm = ChatOpenAI(
		model="gpt-4.1-mini",
		api_key=os.getenv("OPENAI_API_KEY"),
		base_url=os.getenv("OPENAI_API_BASE"),
		temperature=0.7
	)

	# ---- Vague Prompt ----
	vague_prompt = "Write a data privacy policy"
	vague_response = llm.invoke(vague_prompt)

	# ---- Specific Prompt ----
	specific_prompt = (
		"Write a 200-word data privacy policy for European customers "
		"covering GDPR requirements, data retention periods of 30 days, "
		"and user rights for deletion and portability."
	)
	specific_response = llm.invoke(specific_prompt)

	return {
		"vague_prompt": vague_prompt,
		"vague_response": vague_response.content,
		"specific_prompt": specific_prompt,
		"specific_response": specific_response.content
	}


def main():
	print("Zero Shot Prompting Demo")
	print("=" * 50)

	try:
		results = generate_privacy_policies()

		print(f"\nVague Prompt:\n{results['vague_prompt']}")
		print(f"\nVague Response Preview:\n{results['vague_response'][:150]}...\n")

		print("=" * 50)

		print(f"\nSpecific Prompt:\n{results['specific_prompt']}")
		print(f"\nSpecific Response Preview:\n{results['specific_response'][:200]}...\n")

		return results

	except Exception as e:
		print(f"Error occurred: {e}")
		return None


if __name__ == "__main__":
	main()

Zero Shot Prompting Demo

Vague Prompt:
Write a data privacy policy

Vague Response Preview:
**Data Privacy Policy**

**Effective Date:** [Insert Date]

**1. Introduction**  
[Your Company Name] (“we,” “us,” or “our”) is committed to protectin...


Specific Prompt:
Write a 200-word data privacy policy for European customers covering GDPR requirements, data retention periods of 30 days, and user rights for deletion and portability.

Specific Response Preview:
**Data Privacy Policy**

We are committed to protecting the privacy and personal data of our European customers in accordance with the General Data Protection Regulation (GDPR). This policy outlines h...



#### One-Shot Prompting

In [ ]:
print("Creating One-Shot template with example")
print("="*40)

llm = ChatOpenAI(
		model="gpt-4.1-mini",
		api_key=os.getenv("OPENAI_API_KEY"),
		base_url=os.getenv("OPENAI_API_BASE"),
		temperature=0.7
	)

example_policy = """REFUND POLICY \n1. Eligibility: Within 30 days of purchase\n2. Conditions: Product unused and in original packaging\n3. Submit request via support@company.com\n4. Timeline: Refund processed within 5-7 business days\n5. Exceptions: Digital products and custom orders non-refundable"""

print(example_policy)

one_shot_template = PromptTemplate(
	template="""Here's an example of our policy format:
{example}
Now write a {policy_type} policy following this EXACT format with number section:""",
	input_variables=["example", "policy_type"]
)

formatted_prompt = one_shot_template.format(
	example = example_policy,
	policy_type="remote work"
)

response = llm.invoke(formatted_prompt)

has_numbered_sections = any(f"{i}. " in response.content for i in range(1,6))
has_consistent_structure = all(
	keyword in response.content.lower()
	for keyword in ["eligibility", "conditions", "process", "timeline"]
)

if has_numbered_sections and has_consistent_structure:
	print("Follow exact format of the example")
else:
	print("Format need adjustment - check out example")

Creating One-Shot template with example
REFUND POLICY 
1. Eligibility: Within 30 days of purchase
2. Conditions: Product unused and in original packaging
3. Submit request via support@company.com
4. Timeline: Refund processed within 5-7 business days
5. Exceptions: Digital products and custom orders non-refundable
Format need adjustment - check out example


#### Few-Shot Prompting

In [21]:
print("Creating Few-Shot examples")
print("="*50)

llm = ChatOpenAI(
		model="gpt-4.1-mini",
		api_key=os.getenv("OPENAI_API_KEY"),
		base_url=os.getenv("OPENAI_API_BASE"),
		temperature=0.7
	)

examples = [
	{
		"input": "refund request",
		"output": "I understnd you'd like a refund. Let me check the order details. Our refund policy allows returns within 30 days. I'll process this for you right away."
	},
	
	{
		"input": "shipping delay",
		"output": "I apologize for the shipping delay. Let me track your package immediately. I see it's currently in transit and should arrive within 2 days. I'll apply a shipping credit to your account"
	},
	{
		"input": "password reset",
		"output": "I'll help you reset your password. For security, I've sent a reset link to your registered email. The link expires in 1 hour. Please check your spam folder if you don't see it."
	}
]

print("Example loaded")
for i, ex in enumerate(examples, 1):
	print(f"	Example {i}: {ex['input']} -> {ex['output'][:50]}....")

example_prompt = PromptTemplate(
	template="Customer Issuse: {input}\nSupport Response: {output}",
	input_variables=["input", "output"]
)

few_shot_prompt = FewShotPromptTemplate(
	examples=examples,
	example_prompt=example_prompt,
	prefix="You are a helpful customer support agent. Here are examples of how to respond:",
	suffix="Customer Issue: {input}\nSupport Response:",
	input_variables=["input"]
		
)

test_input = "account locked"

formatted_prompt = few_shot_prompt.format(input=test_input)

response_1 = llm.invoke(formatted_prompt)
print(f"AI response: {response_1.content}")


Creating Few-Shot examples
Example loaded
	Example 1: refund request -> I understnd you'd like a refund. Let me check the ....
	Example 2: shipping delay -> I apologize for the shipping delay. Let me track y....
	Example 3: password reset -> I'll help you reset your password. For security, I....
AI response: I see your account is locked. For security reasons, I've sent a verification code to your registered email. Please enter the code to unlock your account. If you need further assistance, I'm here to help.


#### Chain-of-Thought Prompting

In [23]:
from sympy import cot


print("Chain of Thought prompting")
print("="*40)

llm = ChatOpenAI(
		model="gpt-4.1-mini",
		api_key=os.getenv("OPENAI_API_KEY"),
		base_url=os.getenv("OPENAI_API_BASE"),
		temperature=0.7
)

print("Part 1: Direct prompt without chain of thought")
print("-"*40)

direct_prompt = "Our current data retention policy states:\n- Customer data is stored indefitely\n- There is no documented data deletion or review process\n- System backups are retained forever and include personal data\n- No distinction made between active and inactive users\n- Fix this policy to comply with GDPR"

direct_response = llm.invoke(direct_prompt)

print("Part 2: Chain-of-Thought Prompt")
print("-"*40)

reasoning_steps = """Step 1: Review GDRP requirements related to data retention and storage limitation\nStep 2: Identify compliance gaps in the current data retention policy\nStep 3: Reference industry best practices for data retention and deletion\nStep 4: Draft specific, GDPR-compliant policy changes\nStep 5: Propose an implementation and enforcement timeline"""

print(reasoning_steps)

cot_template = PromptTemplate(
    template="""To solve this problem, think through it step-by-step:
{steps}
Problem: {problem}
Now, let's work through each step systematically""",
    input_variables=["steps", "problem"]   
)

cot_prompt = cot_template.format(
        steps=reasoning_steps,
        problem="Fix our data retention policy to comply with GDPR"
    )

print("Applying chain of though reasoning")
print("-"*40)

cot_response = llm.invoke(cot_prompt)
print(f"\n Chain-of-thought response: \n{cot_response.content[:500]}...")


Chain of Thought prompting
Part 1: Direct prompt without chain of thought
----------------------------------------
Part 2: Chain-of-Thought Prompt
----------------------------------------
Step 1: Review GDRP requirements related to data retention and storage limitation
Step 2: Identify compliance gaps in the current data retention policy
Step 3: Reference industry best practices for data retention and deletion
Step 4: Draft specific, GDPR-compliant policy changes
Step 5: Propose an implementation and enforcement timeline
Applying chain of though reasoning
----------------------------------------

 Chain-of-thought response: 
Certainly! Let’s work through each step systematically to fix your data retention policy to comply with GDPR.

---

### Step 1: Review GDPR Requirements Related to Data Retention and Storage Limitation

**Key GDPR principles related to data retention:**

- **Purpose limitation (Article 5(1)(b))**: Personal data must be collected for specified, explicit, and legitim

#### Prompting technique comparison

In [25]:
from email import policy
from unittest import result


llm = ChatOpenAI(
		model="gpt-4.1-mini",
		api_key=os.getenv("OPENAI_API_KEY"),
		base_url=os.getenv("OPENAI_API_BASE"),
		temperature=0.7
	)
test_problem = "Creating an employee remote work policy"

print("Tesing 4 prompting techniques...\n")
results = {}

# 1. ZERO-SHOT prompting
print("Zero Shot Prompting")
print("-"*40)

zero_shot_result = llm.invoke(test_problem)
results['zero_shot'] = zero_shot_result.content
print(f"Preview: {zero_shot_result.content[:100]}...")

# 2. ONE-SHOT PROMPTING
print("One Shot Prompting")
print("-"*40)

single_shot_template = PromptTemplate(
    template="""Example Policy:
VACATION POLICY
1. Eligibility: All full-time employeers
2. Accrual: 15 days per year
3. Request: Submit 2 weeks in advance
4. Approval: Manger discretion
    
Now create: {policy_type}""",
	input_variables=["policy_type"]
)

single_shot_prompt = single_shot_template.format(
	policy_type=test_problem
)

single_shot_result = llm.invoke(single_shot_prompt)
results["single_shot"] = single_shot_result.content
print(f"Review: {single_shot_result.content[:100]}...")

# 3. FEW-SHOT Prompting
print("Few Shot prompting")
print("-"*40)

examples = [
        {"policy": "sick leave", "format": "1. Coverage: 10 days/year\n2. Documentation: Doctor's note after 3 days"},
        {"policy": "training", "format": "1. Budget: $2000/employee/year\n2. Approval: Required for external courses"},
    ]

example_prompt = PromptTemplate(
	template="Policy: {policy}\nFormat:\n{format}",
	input_variables=["policy", "format"]
)

shots_template = FewShotPromptTemplate(
	examples=examples,
	example_prompt=example_prompt,
	prefix="Examples of our policy format:",
	suffix="Now create: {policy_type}",
	input_variables=["policy_type"]
)

formatted_shots_template = shots_template.format(
	policy_type=test_problem
)

shots_result = llm.invoke(formatted_shots_template)
results["few_shot"] = shots_result.content
print(f"Preview: {shots_result.content[:100]}...\n")

# 4. Chain of Thoughts
print("Chain of Thoughts")
print("-"*40)

xot_template = PromptTemplate(
        template="""Think through this step-by-step:
1. Consider who needs remote work
2. Define eligibility criteria
3. Set communication requirements
4. Establish work hours and availability
5. Specify equipment and security needs

Problem: {problem}

Work through each step to create the policy:""",
        input_variables=["problem"]
    )

xot_prompt = xot_template.format(
	problem=test_problem
)

xot_result = llm.invoke(xot_prompt)
results["chain-of-thoughts"] = xot_result.content
print(f"Preview: {xot_result.content[:100]}...\n")

print("Techniques comparison")
print("-"*40)

for technique, response in results.items():
	print(f"\n{technique.upper()}: ")
	print(f" Length: {len(response)} characters")
	print(f" Has Structured: {'numbered' in response.lower() or '1. ' in response}")
	print(f" Specificity: {'employee' in response.lower() and 'remote' in response.lower()}")

lengths = {k:len(v) for k,v in results.items()}
most_detailed = max(lengths, key=lengths.get)

print(f"\n Most Detailed: {most_detailed} ({lengths[most_detailed]} characters)")

Tesing 4 prompting techniques...

Zero Shot Prompting
----------------------------------------
Preview: Creating an effective employee remote work policy is essential for setting clear expectations, maint...
One Shot Prompting
----------------------------------------
Review: REMOTE WORK POLICY

1. Eligibility: All full-time employees who have completed their probationary pe...
Few Shot prompting
----------------------------------------
Preview: Policy: Creating an employee remote work policy  
Format:  
1. Eligibility: Employees must have comp...

Chain of Thoughts
----------------------------------------
Preview: Certainly! Let's work through each step systematically to create a comprehensive employee remote wor...

Techniques comparison
----------------------------------------

ZERO_SHOT: 
 Length: 4793 characters
 Has Structured: True
 Specificity: True

SINGLE_SHOT: 
 Length: 1228 characters
 Has Structured: True
 Specificity: True

FEW_SHOT: 
 Length: 635 characters
 Has Structure